In [1]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install wikipedia


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.8/213.8 kB 5.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.1-py3-none-any.whl size=196978 sha256=1e5814467dfca9196bced6c7b0645e00c193b2c522b97181deb6b549efa18673
  Stored in directory: /Users/perty/Library/Caches/pip/wheels/2d/3c/c7/a1d6bbc0922483b47e7c617104c7a231aa44480d5ce67c4e26
Successfully built pyTelegramBotAPI

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6851fcf37c4a823190b29eb5b7029b729911ff6ca1e64676b10d35a0c99764c4
  Stored in directory: /Users/perty/Library/Caches/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42

Use your knowledge of programming and Python to, with “if” conditionals, control the state of the conversation with the chatbot. For example, when a user starts to chat with your chatbot, the conversation will be in the “0” state. In this state, for example, if the user sends a “hi” message, the chatbot can answer something like “tell me a subject you wanted to talk about.” Then, the status of the conversation will change to “1” and the user will send the subject (for example “databases”) and the chatbot will answer something like “ask me a question about this topic.” Then the status of the conversation will go to “2” and the user will send a question, for example, “what is a database?” Then you will use the Wikipedia library to get a piece of Wikipedia text on databases, or any other topic, to act as the context passed to the Hugging Face pipeline. It will apply the “question-answering” task from Hugging Face and thus provide the answer to the user.

In [12]:
import telebot
from telebot import types
import wikipedia
from transformers import pipeline
from enum import Enum

class State(Enum):
  INITIAL = 'Initial state'
  WAIT_FOR_SUBJECT = 'Bot is waiting for a subject'
  WAIT_FOR_QUESTION = 'Bot is waiting for a question'

token = "TOKEN"
bot = telebot.TeleBot(token, parse_mode=None)
pipe = pipeline("question-answering")

state = State.INITIAL
topic = ' '
wiki_summary = ' '

@bot.message_handler(commands=['start'])
def send_welcome(message):
	bot.reply_to(message, "Hello. I can read Wikipedia and answer questions about a subject.")
 
@bot.message_handler(commands=['help'])
def send_welcome(message):
	bot.reply_to(message, "Select a subject. I will then answer questions on the subject.")
 
@bot.message_handler(func=lambda m: True)
def respond(message: types.Message):
  global state
  global topic
  global wiki_summary

  if state == State.INITIAL:
    bot.reply_to(message, 'Tell me a subject you want to talk about!')
    state = State.WAIT_FOR_SUBJECT
    return

  if state == State.WAIT_FOR_SUBJECT:
    topic = message.text
    search_result = wikipedia.search(topic)
    wiki_summary = wikipedia.summary(search_result[0])
    bot.reply_to(message, 'Ok, ask me a question about ' + topic)
    state = State.WAIT_FOR_QUESTION
    return

  if state == State.WAIT_FOR_QUESTION:
    answer = pipe(question = message.text, context = wiki_summary)
    bot.reply_to(message, answer['answer'])
    bot.send_message(message.chat.id, 'Next topic!')
    state = State.WAIT_FOR_SUBJECT
    return

 


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [13]:
bot.infinity_polling()

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
2022-11-11 13:08:05,210 (__init__.py:964 MainThread) ERROR - TeleBot: "Infinity polling exception: "nil" may refer to: 
zero
NIL (programming language)
Name, Image and Likeness
Nanoimprint lithography
Nomina im Indogermanischen Lexikon
North Island line
Nil (band)
The Nils
Non-Intentional Lifeform
Nil (album)
Nil Recurring
Three Nil
Gob
Gym Class Heroes
Haken
Nil (gi